In [1]:
import pickle
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os
path = '/nfs/team283/aa16/KR_NAS/'
os.chdir(path)
sys.path.append(path)
from IPython.display import Image
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True' + ',dnn.enabled=False'
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

Construct AnnData object:

In [2]:
counts = pd.read_table('data/CRcells_14pcw/ns_tic1219_twobatch/az1219b2_TargetCountMatrix.txt')
genes = counts['TargetName']
counts = counts.drop('TargetName',1)
counts = counts.rename(index=genes)
counts = counts.astype('int')
metadata = pd.read_csv('data/CRcells_14pcw/NS_DSP human foetal brain Cajal-Retzius NanoString profiling 2 annotations.csv')
metadata = metadata.rename(index=metadata['Sanger Sample ID'])
print('Metadata, but no counts, number:', len([c for c in metadata.index if c not in np.array(counts.columns)]))
print('Metadata, but no counts, names:',[c for c in metadata.index if c not in np.array(counts.columns)])
print('Metadata and counts, number:', len([c for c in metadata.index if c in np.array(counts.columns)]))
print('Counts, but no metadata, number:', len([c for c in np.array(counts.columns) if c not in metadata.index]))
metadata = metadata.loc[~pd.isna(metadata.index),:]
metadata = metadata.reindex(np.array(counts.columns))
properties = pd.read_table('data/CRcells_14pcw/ns_tic1219_twobatch/az1219b2_SegmentProperties.txt')
properties = properties.rename(index=properties['Sample_ID'])
properties = properties.reindex(np.array(metadata.index))
properties = properties.drop(labels='Sample_ID', axis = 1)
probe_counts = pd.read_table('data/CRcells_14pcw/ns_tic1219_twobatch/az1219b2_BioProbeCountMatrix.txt')
genes = probe_counts['TargetName']
probe_counts = probe_counts.drop(['TargetName', 'ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol',
                                 'Accessions', 'GenomeBuild', 'GenomicPosition', 'GlobalOutlier', 'GlobalOutlierReason',
                                 'OutlierFrequency', 'OutlierAOIs', 'Gene'],1)
probe_counts = probe_counts.rename(index=genes)
probe_counts = probe_counts.astype('int')
negprobe_counts = probe_counts.loc[probe_counts.index == 'NegProbe-WTX', ]
negprobe_counts.index = negprobe_counts.index + '-' + [str(x) for x in range(len(negprobe_counts.index))]
adataB = sc.AnnData(X=np.array(counts).T, obs=pd.concat((metadata, properties), axis = 1), var=np.array(counts.index))
adataB.var_names = counts.index
adataB.obsm['negProbes'] = negprobe_counts.T
adataB.obs['GeoLOD2.0_01'] = adataB.obs['NegGeoMean_01'] * adataB.obs['NegGeoSD_01']**2
adataB.obs['DetectedGenes2.0'] = np.sum(np.array([adataB.X[i,:] >= adataB.obs['GeoLOD2.0_01'].iloc[i] for i in range(len(adataB.X[:,1]))]), axis = 1)
adataB.var['SYMBOL'] = adataB.var[0]
adataB.var = adataB.var.drop(0, axis = 1)
adataB.var.index = [str(x) for x in range(len(adataB.var.index))]
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 0] = 5
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 1] = 5
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 2] = 5
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 3] = 5
adataB.obs['nuclei'].loc[adataB.obs['nuclei'] == 4] = 5

Metadata, but no counts, number: 42
Metadata, but no counts, names: ['HCA_F_BRA11360456', 'HCA_F_BRA11360459', 'HCA_F_BRA11360462', 'HCA_F_BRA11360465', 'HCA_F_BRA11360468', 'HCA_F_BRA11360471', 'HCA_F_BRA11360474', 'HCA_F_BRA11360477', 'HCA_F_BRA11360480', 'HCA_F_BRA11360483', 'HCA_F_BRA11360486', 'HCA_F_BRA11360489', 'HCA_F_BRA11360492', 'HCA_F_BRA11360495', 'HCA_F_BRA11360498', 'HCA_F_BRA11360501', 'HCA_F_BRA11360504', 'HCA_F_BRA11360507', 'HCA_F_BRA11360510', 'HCA_F_BRA11360513', 'HCA_F_BRA11360516', 'HCA_F_BRA11360519', 'HCA_F_BRA11360522', 'HCA_F_BRA11360525', 'HCA_F_BRA11360528', 'HCA_F_BRA11360531', 'HCA_F_BRA11360534', 'HCA_F_BRA11360537', 'HCA_F_BRA11360540', 'HCA_F_BRA11360543', 'HCA_F_BRA11360546', 'HCA_F_BRA11360549', 'HCA_F_BRA11360552', 'HCA_F_BRA11360555', 'HCA_F_BRA11360558', 'HCA_F_BRA11360561', 'HCA_F_BRA11360564', 'HCA_F_BRA11360567', 'HCA_F_BRA11360570', 'HCA_F_BRA11360573', 'PhiX', 'No match']
Metadata and counts, number: 81
Counts, but no metadata, number: 0


/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [3]:
adataB

AnnData object with n_obs × n_vars = 81 × 18677
    obs: 'Sample_ID', 'Manifest ID', 'slide name', 'scan name', 'panel', 'roi', 'segment', 'aoi', 'area', 'nuclei', 'Type', 'Species', 'Age', 'Background', 'Tissue', 'Slide code', 'Sample slide ID', 'TR', 'Cy5', 'Cy3', 'Pool', 'Collection plate', 'Well', 'PCR plate', 'PCR well', 'Primer plate', 'Primer well', 'I7_Index_ID', 'index', 'I5_Index_ID', 'index2', 'Notes', 'Sanger Sample ID', 'Tag', 'Reads, count', 'Reads, %', 'expected_neg', 'ns_sample_ID', 'RawReads', 'TrimmedReads', 'StitchedReads', 'AlignedReads', 'DeduplicatedReads', 'SequencingSaturation', 'UID', 'NegGeoMean_01', 'NegGeoSD_01', 'GeoLOQ2.5_01', 'NormFactorNeg_01', 'NormFactorQ3', 'NormFactorHK', 'GeoLOD2.0_01', 'DetectedGenes2.0'
    var: 'SYMBOL'
    obsm: 'negProbes'

In [4]:
counts = pd.read_table('data/ns_tic1106_outputs/Run2_TargetCountMatrix.txt')
genes = counts['TargetName']
counts = counts.drop('TargetName',1)
counts = counts.rename(index=genes)
counts = counts.astype('int')
metadata = pd.read_csv('data/NS_DSP CR annotations.csv')
metadata = metadata.rename(index=metadata['Sample_ID'])
print(len([c for c in metadata.index if c not in np.array(counts.columns)]))
print([c for c in metadata.index if c not in np.array(counts.columns)])
print(len([c for c in metadata.index if c in np.array(counts.columns)]))
print(len([c for c in np.array(counts.columns) if c not in metadata.index]))
metadata = metadata.loc[~pd.isna(metadata.index),:]
metadata = metadata.reindex(np.array(counts.columns))
properties = pd.read_table('data/ns_tic1106_outputs/Run2_SegmentProperties.txt')
properties = properties.rename(index=properties['Sample_ID'])
properties = properties.reindex(np.array(metadata.index))
properties = properties.drop(labels='Sample_ID', axis = 1)
probe_counts = pd.read_table('data/ns_tic1106_outputs/Run2_BioProbeCountMatrix.txt')
genes = probe_counts['TargetName']
probe_counts = probe_counts.drop(['TargetName', 'ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol',
                                 'Accessions', 'GenomeBuild', 'GenomicPosition', 'GlobalOutlier', 'GlobalOutlierReason',
                                 'OutlierFrequency', 'OutlierAOIs', 'Gene'],1)
probe_counts = probe_counts.rename(index=genes)
probe_counts = probe_counts.astype('int')
negprobe_counts = probe_counts.loc[probe_counts.index == 'NegProbe-WTX', ]
negprobe_counts.index = negprobe_counts.index + '-' + [str(x) for x in range(len(negprobe_counts.index))]
adata = sc.AnnData(X=np.array(counts).T, obs=pd.concat((metadata, properties), axis = 1), var=np.array(counts.index))
adata.var_names = counts.index
adata.obsm['negProbes'] = negprobe_counts.T
adata.obs['GeoLOD2.0_01'] = adata.obs['NegGeoMean_01'] * adata.obs['NegGeoSD_01']**2
adata.obs['DetectedGenes2.0'] = np.sum(np.array([adata.X[i,:] >= adata.obs['GeoLOD2.0_01'].iloc[i] for i in range(len(adata.X[:,1]))]), axis = 1)
adata.var['SYMBOL'] = adata.var[0]
adata.var = adata.var.drop(0, axis = 1)
adata.var.index = [str(x) for x in range(len(adata.var.index))]
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 0] = 5
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 1] = 5
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 2] = 5
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 3] = 5
adata.obs['nuclei'].loc[adata.obs['nuclei'] == 4] = 5

89
['DSP-1001250001588-C-A02', 'DSP-1001250001588-C-A03', 'DSP-1001250001588-C-A04', 'DSP-1001250001588-C-A05', 'DSP-1001250001588-C-A06', 'DSP-1001250001588-C-A07', 'DSP-1001250001588-C-A08', 'DSP-1001250001588-C-A09', 'DSP-1001250001588-C-A10', 'DSP-1001250001588-C-A11', 'DSP-1001250001588-C-A12', 'DSP-1001250001588-C-B01', 'DSP-1001250001588-C-B03', 'DSP-1001250001588-C-B06', 'DSP-1001250001588-C-B09', 'DSP-1001250001588-C-B12', 'DSP-1001250001588-C-C01', 'DSP-1001250001588-C-C03', 'DSP-1001250001588-C-C06', 'DSP-1001250001588-C-C09', 'DSP-1001250001588-C-C12', 'DSP-1001250001588-C-D01', 'DSP-1001250001588-C-D03', 'DSP-1001250001588-C-D06', 'DSP-1001250001588-C-D09', 'DSP-1001250001588-C-D12', 'DSP-1001250001588-C-E01', 'DSP-1001250001588-C-E03', 'DSP-1001250001588-C-E06', 'DSP-1001250001588-C-E09', 'DSP-1001250001588-C-E12', 'DSP-1001250001588-C-F03', 'DSP-1001250001588-C-F06', 'DSP-1001250001588-C-F09', 'DSP-1001250001588-C-F12', 'DSP-1001250001588-C-G03', 'DSP-1001250001588-C-G06

/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [5]:
adata = anndata.AnnData.concatenate(adata,adataB)

In [6]:
adata

AnnData object with n_obs × n_vars = 226 × 18677
    obs: 'Sample_ID', 'Manifest_ID', 'slide name', 'scan name', 'panel', 'roi', 'nuclei', 'segment', 'aoi', 'area', 'Type', 'Species', 'Age', 'Background', 'Tissue', 'Slide code', 'Sample slide ID', 'TR', 'Cy5', 'Cy3', 'Pool', 'Collection plate', 'Well', 'PCR plate', 'PCR well', 'Primer plate', 'Primer well', 'I7_Index_ID', 'index', 'I5_Index_ID', 'index2', 'indexA', 'indexB', 'lane1', 'lane2', 'totalreads', 'Sanger Sample ID', 'Notes', 'expected_neg', 'ns_sample_ID', 'RawReads', 'TrimmedReads', 'StitchedReads', 'AlignedReads', 'DeduplicatedReads', 'SequencingSaturation', 'UID', 'NegGeoMean_01', 'NegGeoSD_01', 'GeoLOQ2.5_01', 'NormFactorNeg_01', 'NormFactorQ3', 'NormFactorHK', 'GeoLOD2.0_01', 'DetectedGenes2.0', 'Manifest ID', 'Tag', 'Reads, count', 'Reads, %', 'batch'
    var: 'SYMBOL'
    obsm: 'negProbes'

In [7]:
adata.obs['Donor'] = [str(x).split('-')[0] for x in adata.obs['Sample slide ID']]
(adata.obs['Tissue'].astype(str) + ' ' + adata.obs['Pool'].astype(str)
 + ' ' + adata.obs['Age'].astype(str)).value_counts()

Foetal brain (occipital lobe) Residual 19 pcw    20
Foetal brain (temporal lobe) CRcells 19 pcw      20
Foetal brain (occipital lobe) CRcells 19 pcw     20
Foetal brain (temporal lobe) Residual 19 pcw     19
Foetal brain (frontal lobe) CRcells 19 pcw       18
Foetal brain (frontal lobe) Residual 19 pcw      17
Foetal brain (frontal lobe) CRcellsB 14 pcw      16
Foetal brain (occipital lobe) CRcellsB 14 pcw    16
Foetal brain (parietal lobe) CRcells 19 pcw      16
Foetal brain (frontal lobe) Residual 14 pcw      16
Foetal brain (occipital lobe) Residual 14 pcw    16
Foetal brain (parietal lobe) Residual 19 pcw     14
Foetal brain (temporal lobe) Residual 14 pcw      8
Foetal brain (temporal lobe) CRcellsB 14 pcw      8
Foetal brain (occipital lobe) Rings 19 pcw        1
nan Residual nan                                  1
dtype: int64

Save for later use:

In [8]:
dropColumns = (np.sum(pd.isna(adata.obs))[np.sum(pd.isna(adata.obs)) > 10]).index

In [9]:
adata.obs = adata.obs.drop(dropColumns, axis = 1)

In [10]:
adata.obs = adata.obs.astype(str)

In [11]:
adata.write_h5ad('/nfs/team283/aa16/KR_NAS/data/CR_cells_19pcw+14pcw_AnnData.h5ad')

... storing 'slide name' as categorical
... storing 'scan name' as categorical
... storing 'panel' as categorical
... storing 'roi' as categorical
... storing 'nuclei' as categorical
... storing 'Type' as categorical
... storing 'Species' as categorical
... storing 'Age' as categorical
... storing 'Background' as categorical
... storing 'Tissue' as categorical
... storing 'Slide code' as categorical
... storing 'Sample slide ID' as categorical
... storing 'TR' as categorical
... storing 'Cy5' as categorical
... storing 'Cy3' as categorical
... storing 'Pool' as categorical
... storing 'Collection plate' as categorical
... storing 'Well' as categorical
... storing 'PCR plate' as categorical
... storing 'PCR well' as categorical
... storing 'Primer plate' as categorical
... storing 'Primer well' as categorical
... storing 'expected_neg' as categorical
... storing 'RawReads' as categorical
... storing 'TrimmedReads' as categorical
... storing 'StitchedReads' as categorical
... storing 'Al

In [12]:
adata.obs

,Sample_ID,slide name,scan name,panel,roi,nuclei,segment,aoi,area,Type,...,NegGeoMean_01,NegGeoSD_01,GeoLOQ2.5_01,NormFactorNeg_01,NormFactorQ3,NormFactorHK,GeoLOD2.0_01,DetectedGenes2.0,batch,Donor
DSP-1001250001586-D-A02-0,DSP-1001250001586-D-A02,NS_DSP_01CY,NS_DSP_01CY,(v1.0) Human NGS Whole Transcriptome Atlas RNA,1.0,5.0,NS-DSP-01CY-Hob7-frontal-001-ch,NS-DSP-01CY-Hob7-frontal-001-ch-aoi-001,1828.78,FFPE,...,1.34639438653159,1.66161578591248,4.79180146550754,0.519937138094352,0.556903417052606,0.716908881670811,3.717350497118166,3206,0,Hob8
DSP-1001250001586-D-A04-0,DSP-1001250001586-D-A04,NS_DSP_01CY,NS_DSP_01CY,(v1.0) Human NGS Whole Transcriptome Atlas RNA,1.0,118.0,NS-DSP-01CY-Hob7-frontal-001-xbackground,NS-DSP-01CY-Hob7-frontal-001-xbackground-aoi-001,12223.05,FFPE,...,3.34747033169739,1.81460167336544,14.848069926492,1.29269266236476,1.48507577880695,1.5828954228325,11.02248079123203,2087,0,Hob8
DSP-1001250001586-D-A05-0,DSP-1001250001586-D-A05,NS_DSP_01CY,NS_DSP_01CY,(v1.0) Human NGS Whole Transcriptome Atlas RNA,2.0,10.0,NS-DSP-01CY-Hob7-frontal-002-ch,NS-DSP-01CY-Hob7-frontal-002-ch-aoi-001,1858.69,FFPE,...,1.28425764296999,1.58041145756862,4.03252977584088,0.4959417910095,0.556903417052606,0.881037682789155,3.2076907967178085,4252,0,Hob8
DSP-1001250001586-D-A07-0,DSP-1001250001586-D-A07,NS_DSP_01CY,NS_DSP_01CY,(v1.0) Human NGS Whole Transcriptome Atlas RNA,2.0,100.0,NS-DSP-01CY-Hob7-frontal-002-xbackground,NS-DSP-01CY-Hob7-frontal-002-xbackground-aoi-001,8932.28,FFPE,...,1.9656494205436,1.76946792730198,8.18677398148074,0.75907486278745,0.928172361754343,0.886230153786123,6.154481251996513,2758,0,Hob8
DSP-1001250001586-D-A08-0,DSP-1001250001586-D-A08,NS_DSP_01CY,NS_DSP_01CY,(v1.0) Human NGS Whole Transcriptome Atlas RNA,3.0,5.0,NS-DSP-01CY-Hob7-frontal-003-ch,NS-DSP-01CY-Hob7-frontal-003-ch-aoi-001,1086.36,FFPE,...,1.20332493558756,1.44253546312739,3.00745517360771,0.464688006326806,0.371268944701737,0.410616634621005,2.5040091617897,3766,0,Hob8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCA_F_BRA11360569-1,DSP-1001250002203-A-G05,NS_DSP_01FS,NS_DSP_01FS scan 1,(v1.0) Human NGS Whole Transcriptome Atlas RNA,14.0,52.0,NS-DSP-01FS-014-xbackground,NS-DSP-01FS-014-xbackground-aoi-001,17677.87,FFPE,...,4.8613309077246,1.8059158920009,21.3058558177823,1.51640735054543,1.27318697724023,0.919081267648891,15.854415067879055,627,1,Hob5
HCA_F_BRA11360571-1,DSP-1001250002203-A-G06,NS_DSP_01FS,NS_DSP_01FS scan 1,(v1.0) Human NGS Whole Transcriptome Atlas RNA,15.0,31.0,NS-DSP-01FS-015-ch,NS-DSP-01FS-015-ch-aoi-001,3042.34,FFPE,...,2.85894395582267,1.98054487113152,15.7821932683857,0.891797680861044,1.1140386050852,0.911780297624763,11.214373447055003,1261,1,Hob5
HCA_F_BRA11360572-1,DSP-1001250002203-A-G07,NS_DSP_01FS,NS_DSP_01FS scan 1,(v1.0) Human NGS Whole Transcriptome Atlas RNA,15.0,122.0,NS-DSP-01FS-015-xbackground,NS-DSP-01FS-015-xbackground-aoi-001,15047.5,FFPE,...,7.51892378779948,1.55603343147345,22.7092604812955,2.34539707673319,2.2280772101704,2.18185156459167,18.20511933169877,2390,1,Hob5
HCA_F_BRA11360574-1,DSP-1001250002203-A-G08,NS_DSP_01FS,NS_DSP_01FS scan 1,(v1.0) Human NGS Whole Transcriptome Atlas RNA,16.0,37.0,NS-DSP-01FS-016-ch,NS-DSP-01FS-016-ch-aoi-001,3272.12,FFPE,...,2.70937357776479,1.79462349909142,11.6896704155734,0.845141811302684,0.954890232930173,1.04496307769191,8.726007692966029,2607,1,Hob5


In [13]:
adata.obs['nuclei']

DSP-1001250001586-D-A02-0      5.0
DSP-1001250001586-D-A04-0    118.0
DSP-1001250001586-D-A05-0     10.0
DSP-1001250001586-D-A07-0    100.0
DSP-1001250001586-D-A08-0      5.0
                             ...  
HCA_F_BRA11360569-1           52.0
HCA_F_BRA11360571-1           31.0
HCA_F_BRA11360572-1          122.0
HCA_F_BRA11360574-1           37.0
HCA_F_BRA11360575-1          121.0
Name: nuclei, Length: 226, dtype: category
Categories (125, object): ['5.0', '6.0', '7.0', '8.0', ..., '293.0', '313.0', '317.0', 'nan']

In [14]:
np.sum(adata.obs['nuclei'] == '0.0')

0

In [15]:
np.sum(adata.obs['nuclei'] == '0.0')

0